In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import linear, relu, sigmoid
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score

In [ ]:
# load dataset
# Set the path to the main folder containing the CT scan folders
folder_path = "ct_scans/"

# Define the categories and their corresponding labels
categories = ["benign", "malignant", "normal"]
labels = [0, 1, 0]

# Initialize empty lists to store the images and labels
X = []
y = []

# Loop through the categories
for category, label in zip(categories, labels):
    # Set the path to the current category folder
    category_path = os.path.join(folder_path, category)
    
    # Loop through the files in the current category folder
    for file_name in os.listdir(category_path):
        # Set the path to the current file
        file_path = os.path.join(category_path, file_name)
        
        # Load the image in RGB format
        image = cv2.imread(file_path, cv2.IMREAD_COLOR)
        
        # Append the image and its corresponding label to the lists
        X.append(image)
        y.append(label)

# Convert the lists to NumPy arrays for further processing
X = np.array(X)
y = np.array(y)

print ('The shape of X is: ' + str(X.shape))
print ('The shape of y is: ' + str(y.shape))

In [ ]:
# Split the data into training, validation, and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("Training set size:", len(X_train))
print("Test set size:", len(X_test))
print("Validation set size:", len(X_val))

In [ ]:
# Display the first 5 images
for i in range(5):
    plt.imshow(X_train[i])
    plt.title(f"Label: {y_train[i]}")
    plt.axis('off')
    plt.show()

In [ ]:
# Normalize the image data to a range of 0 to 1
X_train = X_train.astype("float32") / 255.0
X_val = X_val.astype("float32") / 255.0
X_test = X_test.astype("float32") / 255.0

In [ ]:
# Build the CNN model
tf.random.set_seed(1234)
model = Sequential(
    [
        tf.keras.Input(shape=(224, 224, 3)),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, (3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(2, activation="softmax"),
    ]
)

model.summary()

In [ ]:
# Compile the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Training the model
model.fit(X_train, y_train, epochs=40)

In [ ]:
# Evaluate the model on the testing set
loss, accuracy = model.evaluate(X_test, y_test)

# Print the accuracy
print(f"Accuracy on the testing set: {accuracy * 100:.2f}%")

In [ ]:
# Train the model and store the training history
history = model.fit(X_train, y_train, batch_size=32, epochs=40)

# Extract the accuracy values from the training history
train_accuracy = history.history['accuracy']

# Create an array of the epoch numbers
epochs = range(1, len(train_accuracy) + 1)

# Plot the accuracy values
plt.plot(epochs, train_accuracy, 'b', label='Train Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
for i in range(len(X_test)):
    image = X_test[i]
    label = y_test[i]

    # Display the image
    plt.imshow(image)
    plt.title(f"Label: {label}")
    plt.axis('off')
    plt.show()